In [19]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.nn import Transformer
from tqdm import tqdm
from torchtext.data.metrics import bleu_score

import torchtext

import gc
import copy

from torch.utils.data import DataLoader

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker


from dataset import nmtDataset
import helpers as utils

import spacy
import numpy as np

import random
import math
import time

In [20]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [21]:
log = utils.Logger('logs/transformers.out')  

In [22]:
hyp_params = {
    "batch_size": 128,
    "lr": 0.0005,
    "num_epochs": 10,
    "d_model": 512,
    "n_head": 8,
    "num_encoder_layers": 3,
    "num_decoder_layers": 3,
    "feedforward_dim": 256,
    "dropout": 0.1
}

In [23]:
nmtds_train = nmtDataset('datasets/Multi30k/', 'train')
nmtds_valid = nmtDataset('datasets/Multi30k/', 'val', nmtds_train)
nmtds_test = nmtDataset('datasets/Multi30k/', 'test', nmtds_train)

SRC_PAD_IDX = nmtds_train.src_vocab["<pad>"]
TRG_PAD_IDX = nmtds_train.trg_vocab["<pad>"]

train_dataloader = DataLoader(nmtds_train, batch_size=hyp_params["batch_size"], shuffle=True,
                              collate_fn=lambda batch_size: utils.collate_fn(batch_size, SRC_PAD_IDX, device))

valid_dataloader = DataLoader(nmtds_valid, batch_size=hyp_params["batch_size"], shuffle=True,
                              collate_fn=lambda batch_size: utils.collate_fn(batch_size, SRC_PAD_IDX, device))

hyp_params["src_vocab_size"] = len(nmtds_train.src_vocab)
hyp_params["trg_vocab_size"] = len(nmtds_train.trg_vocab)

In [24]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout, maxlen = 5000):
        super(PositionalEncoding, self).__init__()
        
        # A tensor consists of all the possible positions (index) e.g 0, 1, 2, ... max length of input
        # Shape (pos) --> [max len, 1]
        pos = torch.arange(0, maxlen).unsqueeze(1)
        
        pos_encoding = torch.zeros((maxlen, d_model))
        
        # In the paper, they had 2i in the positional encoding formula
        # where i is the dimension 
        sin_den = 10000 ** (torch.arange(0, d_model, 2)/d_model) # sin for even item of position's dimension
        cos_den = 10000 ** (torch.arange(1, d_model, 2)/d_model) # cos for odd 
        
        pos_encoding[:, 0::2] = torch.sin(pos / sin_den) 
        pos_encoding[:, 1::2] = torch.cos(pos / cos_den)
        
        # Shape (pos_embedding) --> [max len, d_model]
        pos_encoding = pos_encoding.unsqueeze(-2)
        # Shape (pos_embedding) --> [max len, 1, d_model]

        self.dropout = nn.Dropout(dropout)
        self.register_buffer('pos_encoding', pos_encoding)

    def forward(self, token_embedding):
        
        # shape (token_embedding) --> [sentence len, batch size, d_model]
        
        # Combining embeddings with positional encodings
        # Note: As we made positional encoding with the size max length of sentence in our dataset 
        #       hence here we are picking till the sentence length in a batch
        #       Another thing to notice is in the paper they used FIXED positional encoding, there are
        #       methods where we can also learn them but we are doing as presented in the paper
        return self.dropout(token_embedding + self.pos_encoding[:token_embedding.size(0), :])
    
class InputEmbedding(nn.Module):
    def __init__(self, vocab_size, d_model):
        super(InputEmbedding, self).__init__()
        
        self.embedding = nn.Embedding(vocab_size, d_model)
        self.d_model = d_model

    def forward(self, tokens):
        # shape (tokens) --> [sentence len, batch size]
        # shape (inp_emb) --> [sentence len, batch size, d_model]
        inp_emb = self.embedding(tokens.long()) * math.sqrt(self.d_model)
        return inp_emb

In [25]:
class Seq2SeqTransformer(nn.Module):
    def __init__(self, 
                 src_vocab_size, 
                 trg_vocab_size, 
                 d_model, 
                 dropout,
                 nhead,
                 num_encoder_layers,
                 num_decoder_layers,
                 dim_feedforward,
                 src_pad_idx,
                 trg_pad_idx
                ):
        super(Seq2SeqTransformer, self).__init__()
        
        self.src_pad_idx = src_pad_idx
        self.trg_pad_idx = trg_pad_idx
        
        self.src_inp_emb = InputEmbedding(src_vocab_size, d_model)
        self.trg_inp_emb = InputEmbedding(trg_vocab_size, d_model)
        
        self.positional_encoding = PositionalEncoding(d_model, dropout=dropout)
        
        self.transformer = Transformer(d_model=d_model,
                                       nhead=nhead,
                                       num_encoder_layers=num_encoder_layers,
                                       num_decoder_layers=num_decoder_layers,
                                       dim_feedforward=dim_feedforward,
                                       dropout=dropout)
        
        self.generator = nn.Linear(d_model, trg_vocab_size)
    
    def forward(self, src, trg):
        src_emb = self.positional_encoding(self.src_inp_emb(src))
        trg_emb = self.positional_encoding(self.trg_inp_emb(trg))
        
        src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = self.create_mask(src, trg)
        
        outs = self.transformer(src = src_emb, 
                                tgt = trg_emb, 
                                src_mask = src_mask,
                                tgt_mask = tgt_mask, 
                                src_key_padding_mask = src_padding_mask, 
                                tgt_key_padding_mask = tgt_padding_mask,
                                memory_key_padding_mask = src_padding_mask
                               )
        return self.generator(outs)
        

    def create_mask(self, src, trg):
        src_seq_len = src.shape[0]
        trg_seq_len = trg.shape[0]

        trg_mask = self.transformer.generate_square_subsequent_mask(trg_seq_len).to(device)
        src_mask = torch.zeros((src_seq_len, src_seq_len),device=device).type(torch.bool) # All False hence unchanged

        # Padding masking will allow attention to ignore padding <pad> tokens 
        src_padding_mask = (src == self.src_pad_idx).transpose(0, 1)
        trg_padding_mask = (trg == self.trg_pad_idx).transpose(0, 1)
        
        return src_mask, trg_mask, src_padding_mask, trg_padding_mask
    
    def encode(self, src):
        src_mask = torch.zeros((src.shape[0], src.shape[0]),device=device).type(torch.bool)
        src_padding_mask = (src == self.src_pad_idx).transpose(0, 1)
        
        return self.transformer.encoder(self.positional_encoding(self.src_inp_emb(src)), src_mask, src_padding_mask)

    def decode(self, trg, memory):
        trg_seq_len = trg.shape[0]
        trg_mask = self.transformer.generate_square_subsequent_mask(trg_seq_len).type(torch.bool).to(device)
        trg_padding_mask = (trg == self.trg_pad_idx).transpose(0, 1)
        
        return self.transformer.decoder(tgt = self.positional_encoding(self.trg_inp_emb(trg)), 
                                        memory = memory, 
                                        tgt_mask = trg_mask,
                                        tgt_key_padding_mask = trg_padding_mask
                                       )

In [26]:
def train_model(model, train_dataloader, criterion, optimizer):
    model.train()
    epoch_loss = 0
    for batch_idx, batch in enumerate(tqdm(train_dataloader)):
        # shape (src, trg) --> [seq len, batch size]
        src = batch["src"]
        trg = batch["trg"]

        # Clear the accumulating gradients
        optimizer.zero_grad()

        # shape (trg_inp, trg_out) --> [seq len - 1, batch size]
        trg_inp = trg[:-1, :]
        trg_out = trg[1:, :]

        # shape --> (seq len - 1) * batch size 
        # Making all target seqeunces in 1d tensor
        trg_out = trg_out.reshape(-1)

        # shape (logits) --> [seq len - 1, batch size, trg vocab size]
        logits = model(src, trg_inp)

        # shape (logits) --> [(seq len - 1) * batch size, trg vocab size]
        logits = logits.reshape(-1, logits.shape[-1])

        loss = criterion(logits, trg_out)

        loss.backward()

        optimizer.step()
        epoch_loss += loss.detach().cpu()

    return epoch_loss/len(train_dataloader)

def evaluate_model(model, valid_dataloader, criterion):
    model.eval()
    epoch_loss = 0
    with torch.no_grad():
        for batch_idx, batch in enumerate(valid_dataloader):
            # shape (src, trg) --> [seq len, batch size]
            src = batch["src"]
            trg = batch["trg"]

            # shape (trg_inp, trg_out) --> [seq len - 1, batch size]
            trg_inp = trg[:-1, :]
            trg_out = trg[1:, :]

            # shape --> (seq len - 1) * batch size 
            # Making all target seqeunces in 1d tensor
            trg_out = trg_out.reshape(-1)

            # shape (logits) --> [seq len - 1, batch size, trg vocab size]
            logits = model(src, trg_inp)

            # shape (logits) --> [(seq len - 1) * batch size, trg vocab size]
            logits = logits.reshape(-1, logits.shape[-1])

            loss = criterion(logits, trg_out)

            epoch_loss += loss.detach().cpu()
    
    return epoch_loss/len(valid_dataloader)

In [27]:
model = Seq2SeqTransformer(hyp_params["src_vocab_size"],
                                hyp_params["trg_vocab_size"],
                                hyp_params["d_model"],
                                hyp_params["dropout"],
                                hyp_params["n_head"],
                                hyp_params["num_encoder_layers"],
                                hyp_params["num_decoder_layers"],
                                hyp_params["feedforward_dim"],
                                SRC_PAD_IDX,
                                TRG_PAD_IDX
                                ).to(device)

for p in model.parameters():
    if p.dim() > 1:
        nn.init.xavier_uniform_(p)

criterion = nn.CrossEntropyLoss(ignore_index=TRG_PAD_IDX)

optimizer = optim.Adam(model.parameters(), lr=hyp_params["lr"])

In [28]:
min_el = math.inf
patience = 1
best_model = {}
best_epoch = 0

epoch_loss = 0
for epoch in range(hyp_params["num_epochs"]):
    start = time.time()
    
    gc.collect()
    torch.cuda.empty_cache()
    
    epoch_loss = train_model(model, train_dataloader, criterion, optimizer)
    eval_loss = evaluate_model(model, valid_dataloader, criterion)
    
    log.log(f"Epoch: {epoch+1}, Train loss: {epoch_loss}, Eval loss: {eval_loss}, patience: {patience}. Time {time.time() - start}")

    
    if eval_loss < min_el:
        best_epoch = epoch+1
        min_el = eval_loss
        best_model = copy.deepcopy(model)
        torch.save({
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'eval_loss': min_el
        }, 'model-transformer.pt')
        patience = 1
    else:
        patience += 1
    
    if patience == 10:
        log.log("[STOPPING] Early stopping in action..")
        log.log(f"Best epoch was {best_epoch} with {min_el} eval loss")
        break
        
log.log(f"Best epoch was {best_epoch} with {min_el} eval loss")
log.close()

100%|██████████| 227/227 [01:15<00:00,  3.03it/s]


Epoch: 1, Train loss: 4.186300277709961, Eval loss: 2.948056697845459, patience: 1. Time 76.640953540802


100%|██████████| 227/227 [01:15<00:00,  3.01it/s]


Epoch: 2, Train loss: 2.5421230792999268, Eval loss: 2.004237651824951, patience: 1. Time 76.4445264339447


100%|██████████| 227/227 [01:15<00:00,  3.02it/s]


...

Epoch: 8, Train loss: 0.7432900071144104, Eval loss: 1.4157357215881348, patience: 2. Time 76.57517194747925


100%|██████████| 227/227 [01:15<00:00,  3.02it/s]


Epoch: 9, Train loss: 0.6643882393836975, Eval loss: 1.4319831132888794, patience: 3. Time 76.32783889770508


100%|██████████| 227/227 [01:15<00:00,  3.00it/s]


Epoch: 10, Train loss: 0.5918142199516296, Eval loss: 1.4792366027832031, patience: 4. Time 76.65888452529907
Best epoch was 6 with 1.373586654663086 eval loss


In [29]:
model_l = Seq2SeqTransformer(hyp_params["src_vocab_size"],
                                hyp_params["trg_vocab_size"],
                                hyp_params["d_model"],
                                hyp_params["dropout"],
                                hyp_params["n_head"],
                                hyp_params["num_encoder_layers"],
                                hyp_params["num_decoder_layers"],
                                hyp_params["feedforward_dim"],
                                SRC_PAD_IDX,
                                TRG_PAD_IDX
                                ).to(device)

model_l.load_state_dict(torch.load('model-transformer.pt', map_location=device)["model_state_dict"])
model_l.eval()

Seq2SeqTransformer(
  (src_inp_emb): InputEmbedding(
    (embedding): Embedding(5893, 512)
  )
  (trg_inp_emb): InputEmbedding(
    (embedding): Embedding(7853, 512)
  )
  (positional_encoding): PositionalEncoding(
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (transformer): Transformer(
    (encoder): TransformerEncoder(
      (layers): ModuleList(
        (0): TransformerEncoderLayer(
          (self_attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=512, out_features=512, bias=True)
          )
          (linear1): Linear(in_features=512, out_features=256, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
          (linear2): Linear(in_features=256, out_features=512, bias=True)
          (norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (norm2): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (dropout1): Dropout(p=0.1, inplace=False)
          (dropout2): Dropout(p=0.1, inplace=False)

In [30]:
def translate(snt, dataset, model, device):
    snt = torch.tensor(snt).view(-1,1).to(device)
    
    num_tokens = snt.shape[0]
    max_len = 50
    
    with torch.no_grad():
        memory = model.encode(snt).to(device)
    
    ys = torch.LongTensor([dataset.trg_vocab['<sos>']]).unsqueeze(0).to(device)

    for i in range(max_len):
        with torch.no_grad():
            out = model.decode(ys, memory)
            
        out = out.transpose(0, 1)
        prob = model.generator(out[:, -1])
        next_word = prob.argmax().detach().item()

        ys = torch.cat([ys, torch.tensor([next_word]).unsqueeze(1).to(device)], dim=0)
        
        
        if next_word == dataset.trg_vocab['<eos>']:
            break
    
    return dataset.trg_vocab.lookup_tokens(ys.squeeze().cpu().numpy())


def bleu(model, dataset, device):
    targets = []
    outputs = []

    for example in tqdm(dataset):
        src = example["src"]
        trg = example["trg"]
        
        trg = dataset.trg_vocab.lookup_tokens(trg)    
        prediction = translate(src, dataset, model, device)
        
        prediction = prediction[1:-1]  # removing <sos> and <eos> tokens
        trg = trg[1:-1]
        
        targets.append([trg])
        outputs.append(prediction)

    return bleu_score(outputs, targets)


In [31]:
bleu(model_l, nmtds_test, device)

100%|██████████| 1000/1000 [00:57<00:00, 17.29it/s]


0.3632104694843292